In [ ]:
from pathlib import Path

import Metashape

from mynd.backends import metashape as backend
from mynd.utils.log import logger
from mynd.utils.result import Ok, Err, Result

### Define workflow for obtaining rectification results and image pairs from Metashape

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import tqdm

from mynd.data.image import Image, ImagePair

from mynd.backends.metashape.data_types import (
    SensorPair,
    CameraPair,
    StereoGroup,
)

from mynd.backends.metashape.camera_helpers import (
    compute_stereo_calibration,
    get_stereo_groups,
)

from mynd.backends.metashape.image_helpers import (
    ImagePairLoader,
    generate_image_loaders,
)

from mynd.geometry.stereo import (
    StereoCalibration,
    RectificationResult,
    compute_stereo_rectification,
    rectify_image_pair,
)


def generate_stereo_rectification(
    chunk: Metashape.Chunk,
) -> tuple[RectificationResult, list[ImagePairLoader]]:
    """Returns rectification results and image loaders for the stereo groups in a Metashape chunk."""

    # Get pairs of sensors and cameras
    stereo_groups: list[StereoGroup] = get_stereo_groups(chunk)

    # NOTE: Process the first stereo group for now
    group: StereoGroup = stereo_groups[0]

    # Compute the stereo calibration for the sensor pair
    calibration: StereoCalibration = compute_stereo_calibration(group.sensor_pair)

    # Compute homographies, image transformations and camera matrices for stereo rectification
    rectification: RectificationResult = compute_stereo_rectification(calibration)

    # Set up image loaders for all camera pairs
    image_loaders: list[ImagePairLoader] = generate_image_loaders(group.camera_pairs)

    return rectification, image_loaders

### Load a ONNX model from file

In [ ]:
import os

import cv2
import tqdm

from mynd.io import write_image
from mynd.geometry.hitnet import (
    HitnetConfig,
    load_hitnet,
    compute_disparity,
)
from mynd.geometry.range_maps import (
    compute_range_from_disparity,
    compute_normals_from_range,
)


def estimate_stereo_geometry_and_export(
    rectification: RectificationResult,
    image_loaders: list[ImagePairLoader],
    model: HitnetConfig,
    directories: dict[str, Path],
) -> None:
    """Function for developing Hitnet functionality."""

    focal_length: float = rectification.master.calibration.focal_length
    baseline: float = rectification.slave.location[0]

    for loader in tqdm.tqdm(image_loaders, desc="Estimating disparity..."):
        images: ImagePair = loader()

        rectified_images: tuple = rectify_image_pair(
            master_image=images.first.to_array(),
            slave_image=images.second.to_array(),
            rectification=rectification,
        )

        left_rect: Image = Image(data=rectified_images[0], format=images.first.format)
        right_rect: Image = Image(data=rectified_images[1], format=images.second.format)

        disparity_maps: tuple[np.ndarray, np.ndarray] = compute_disparity(
            model, left=left_rect, right=right_rect
        )

        left_ranges: np.ndarray = compute_range_from_disparity(
            disparity=disparity_maps[0],
            baseline=baseline,
            focal_length=focal_length,
        )

        right_ranges: np.ndarray = compute_range_from_disparity(
            disparity=disparity_maps[1],
            baseline=baseline,
            focal_length=focal_length,
        )

        left_normals: np.ndarray = compute_normals_from_range(
            range_map=left_ranges,
            camera_matrix=rectification.master.calibration.camera_matrix,
            flipped=True,
        )

        right_normals: np.ndarray = compute_normals_from_range(
            range_map=right_ranges,
            camera_matrix=rectification.slave.calibration.camera_matrix,
            flipped=True,
        )

        # Convert to 16 bit to save storage space
        left_ranges: np.ndarray = left_ranges.astype(np.float16)
        right_ranges: np.ndarray = right_ranges.astype(np.float16)
        left_normals: np.ndarray = left_normals.astype(np.float16)
        right_normals: np.ndarray = right_normals.astype(np.float16)

        paths: dict[str, Path] = {
            "left_ranges": directories.get("ranges") / f"{images.first.label}.tiff",
            "right_ranges": directories.get("ranges") / f"{images.second.label}.tiff",
            "left_normals": directories.get("normals") / f"{images.first.label}.tiff",
            "right_normals": directories.get("normals") / f"{images.second.label}.tiff",
        }

        write_image(uri=paths.get("left_ranges"), image=left_ranges)
        write_image(uri=paths.get("right_ranges"), image=right_ranges)
        write_image(uri=paths.get("left_normals"), image=left_normals)
        write_image(uri=paths.get("right_normals"), image=right_normals)

In [ ]:
from mynd.runtime import Environment, load_environment

PATHS: dict[str, Path] = {
    "DOCUMENT_IN": Path(
        "/data/kingston_snv_01/acfr_revisits_metashape_projects/r23685bc_working_version.psz"
    ),
    "DOCUMENT_OUT": Path(
        "/data/kingston_snv_01/acfr_revisits_metashape_projects_test/r23685bc_working_version_saved.psz"
    ),
    "CACHE": Path("/home/martin/dev/benthoscan/.cache/"),
}


result: Result[Environment, str] = load_environment()

match result:
    case Ok(environment):
        logger.info("loaded environment")
    case Err(message):
        logger.error(message)

result: Result[str, str] = backend.load_project(PATHS.get("DOCUMENT_IN"))
match result:
    case Ok(message):
        logger.info(message)
    case Err(message):
        logger.error(message)

document: Metashape.Document = backend.context._backend_data.get("document")

target_labels: list[str] = ["r23685bc_20100605_021022"]
target_chunks: list[Metashape.Chunk] = [
    chunk for chunk in document.chunks if chunk.label in target_labels
]

directories: dict[str, Path] = {
    "ranges": Path("/data/kingston_snv_01/stereo_range_maps"),
    "normals": Path("/data/kingston_snv_01/stereo_normal_maps"),
}

# Load model
model: HitnetConfig = load_hitnet(
    environment.resource_directory / Path("hitnet_models/hitnet_eth3d_720x1280.onnx")
)

# Generate range maps based on stereo pairs
for chunk in target_chunks:

    output_directories = {
        key: path / f"{chunk.label}" for key, path in directories.items()
    }

    rectification, image_loaders = generate_stereo_rectification(chunk)

    # NOTE: Test Hitnet workflow - Development purposes only!
    estimate_stereo_geometry_and_export(
        rectification, image_loaders, model, output_directories
    )